### 9.3 고속 퓨리에 변환 (fft)

수식을 짝수부와 홀수부로 분리 (N =2L)

g(k)로 g(k+L)의 값을 찾을 수 있음

순서 : 홀수부와 짝수부로 분해 - 2개가 하나의 세트가 될 때까지 분해 - dft 계산 - 병합 - 최종 결과 출력됨


zero padding = 전체 이미지 사이즈를 2^n개로 만들어서 계산을 수월하게 하도록 하기 위해

In [2]:
# 9.3.1 고속 푸리에 변환
import numpy as np, cv2
from Common.dft2d import dft, idft, calc_spectrum, fftshift, exp
from Common.fft2d import zeropadding

def butterfly(pair, L, N, dir):
    for k in range(L):
        Geven, Godd = pair[k], pair[k + L]
        pair[k] = Geven + Godd * exp(dir * k / N) # 짝수부
        pair[k+L] = Geven - Godd * exp(dir * k / N) # 홀수부

def pairing(g, N, dir, start=0, stride=1):
    if N == 1: return[g[start]]
    L = N // 2
    sd = stride * 2
    part1 = pairing(g, L, dir, start, sd) # 홀수 신호 처리
    part2 = pairing(g, L, dir, start + stride, sd) # 짝수 신호 처리
    pair = part1 + part2
    butterfly(pair, L, N, dir)
    return pair

def fft(g):
    return pairing(g, len(g), 1)

def ifft(g):
    fft = pairing(g, len(g), -1)
    return [v / len(g) for v in fft]

def fft2(image):
    pad_img = zeropadding(image)
    tmp = [fft(row) for row in pad_img]
    dst = [fft(row) for row in np.transpose(tmp)]
    return np.transpose[dst]

def ifft2(image):
    tmp = [fft(row) for row in image]
    dst = [fft(row) for row in np.transpose(tmp)]
    return np.transpose[dst]

image = cv2.imread("images_09/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

# dft1 = fft2(image)
dft2 = np.fft.fft2(image)
dft3 = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)

# spectrum1 = calc_spectrum(fftshift(dft1))
spectrum2 = calc_spectrum(fftshift(dft2))
spectrum3 = calc_spectrum(fftshift(dft3))

# idft1 = fft2(dft1).real
# idft2 = np.fft.ifft2(dft1).real
idft3 = cv2.idft(dft3, flags=cv2.DFT_SCALE)[:, :, 0]

# print('user 방법 변환 행렬 크기:', dft1.shape)
# print('np 방법 변환 행렬 크기:', dft2.shape)
print('cv 방법 변환 행렬 크기:', dft3.shape)

# cv2.imshow('spectrum1', spectrum1)
# cv2.imshow('spectrum2 np', spectrum2)
cv2.imshow('spectrum3 cv', spectrum3)
# cv2.imshow('idft_img1', cv2.convertScaleAbs(idft1))
# cv2.imshow('idft_img2', cv2.convertScaleAbs(idft2))
cv2.imshow('idft_img3', cv2.convertScaleAbs(idft3))
cv2.waitKey(0)
    

cv 방법 변환 행렬 크기: (852, 1280, 2)
